In [1]:
import tinygrad
from tinygrad import Tensor

In [2]:
import torch

In [3]:
t1 = Tensor([1, 2, 3, 4, 5])
t2 = torch.Tensor([1, 2, 3, 4, 5])

In [4]:
t1.shape

(5,)

In [5]:
print(list(Tensor.full((2,2), 5)))
print(Tensor.full((2,2), 5).numpy())

[<Tensor <LB CLANG (2,) int ShapeTracker(views=(View(shape=(2,), strides=(0,), offset=0, mask=None, contiguous=False),))> on CLANG with grad None>, <Tensor <LB CLANG (2,) int ShapeTracker(views=(View(shape=(2,), strides=(0,), offset=0, mask=None, contiguous=False),))> on CLANG with grad None>]
[[5 5]
 [5 5]]


In [6]:
t1, t2

(<Tensor <LB CLANG (5,) int (<MetaOps.COPY: 3>, None)> on CLANG with grad None>,
 tensor([1., 2., 3., 4., 5.]))

In [7]:
t1 + 1

<Tensor <LB CLANG (5,) int (<BinaryOps.ADD: 1>, None)> on CLANG with grad None>

In [8]:
t1.numpy()

array([1, 2, 3, 4, 5], dtype=int32)

In [9]:
import tinygrad.nn as nn

In [10]:
class TinyNet:
    def __init__(self):
        self.layer1 = nn.Linear(784, 128)
        self.layer2 = nn.Linear(128, 10)
    def __call__(self, X:Tensor):
        X = self.layer1(X).leakyrelu()
        X = self.layer2(X).log_softmax()
        return X

In [11]:
net = TinyNet()
net

In [12]:
from tinygrad.nn.state import get_parameters
get_parameters(net)

[<Tensor <LB CLANG (128, 784) float (<BinaryOps.ADD: 1>, None)> on CLANG with grad None>,
 <Tensor <LB CLANG (128,) float (<BinaryOps.ADD: 1>, None)> on CLANG with grad None>,
 <Tensor <LB CLANG (10, 128) float (<BinaryOps.ADD: 1>, None)> on CLANG with grad None>,
 <Tensor <LB CLANG (10,) float (<BinaryOps.ADD: 1>, None)> on CLANG with grad None>]

In [13]:
from extra.datasets import fetch_mnist
import numpy as np

ModuleNotFoundError: No module named 'extra'

In [ ]:
X_train, Y_train, X_test, Y_test = fetch_mnist()

In [ ]:
from tinygrad.nn.optim import SGD
opt = SGD(get_parameters(net), lr=3e-4)

with Tensor.train():
  for step in range(100):
    # random sample a batch
    samp = np.random.randint(0, X_train.shape[0], size=(64))
    batch = Tensor(X_train[samp], requires_grad=False)
    # get the corresponding labels
    labels = Tensor(Y_train[samp])

    # forward pass
    out = net(batch)

    # compute loss
    # loss = sparse_categorical_crossentropy(out, labels)
    loss = out.sparse_categorical_crossentropy(labels)

    # zero gradients
    opt.zero_grad()

    # backward pass
    loss.backward()

    # update parameters
    opt.step()

    # calculate accuracy
    pred = out.argmax(axis=-1)
    acc = (pred == labels).mean()

    if step % 100 == 0:
      print(f"Step {step+1} | Loss: {loss.numpy()} | Accuracy: {acc.numpy()}")

In [ ]:
from tinygrad.helpers import Timing
with Timing("Time: "):
  avg_acc = 0
  for step in range(1000):
    # random sample a batch
    samp = np.random.randint(0, X_test.shape[0], size=(64))
    batch = Tensor(X_test[samp], requires_grad=False)
    # get the corresponding labels
    labels = Y_test[samp]

    # forward pass
    out = net(batch)

    # calculate accuracy
    pred = out.argmax(axis=-1).numpy()
    avg_acc += (pred == labels).mean()
  print(f"Test Accuracy: {avg_acc / 1000}")

In [ ]:
from tinygrad.nn.state import safe_save, safe_load, get_state_dict, load_state_dict

# first we need the state dict of our model
state_dict = get_state_dict(net)
print(state_dict)
print("-------------------------------------")

# then we can just save it to a file
safe_save(state_dict, "model.safetensors")

# and load it back in
state_dict = safe_load("model.safetensors")
print(state_dict)
print("-------------------------------------")
load_state_dict(net, state_dict)

In [ ]:
state_dict

In [ ]:
t1.dtype.vec(2)

In [ ]:
t1.dtype.__repr__(), t1.dtype, t1.dtype.scalar()

In [ ]:
t1 = t1.float()
t1.numpy()

In [ ]:
t1.dtype.vec(5), t1.dtype.scalar()

In [ ]:
t1.is_floating_point()

In [ ]:
from tinygrad import dtypes

In [ ]:
t01 = Tensor(1.)
dtypes.is_int(t01.dtype)

In [ ]:
t01, t01.numpy()

In [ ]:
t1.dtype.scalar()

In [ ]:
dtypes.finfo(t01.dtype)

# Broadcasting

In [14]:
import tinygrad
from tinygrad import Tensor
import torch
import numpy as np

In [15]:
ex1 = [1.,2.,3.]
ex2 = [2.]
t1 = Tensor(ex1)
t2 = Tensor(ex2)
t01 = torch.Tensor(ex1)
t02 = torch.Tensor(ex2)
n1 = np.array(ex1)
n2 = np.array(ex2)

In [16]:
(t1*t2).numpy(), t01*t02, n1*n2

(array([2., 4., 6.], dtype=float32), tensor([2., 4., 6.]), array([2., 4., 6.]))

In [17]:
ex1 = [[1.,2.,3.],
      [4.,5.,6.],
      [7.,8.,9.]]
ex2 = [2.]
t1 = Tensor(ex1)
t2 = Tensor(ex2)
t01 = torch.Tensor(ex1)
t02 = torch.Tensor(ex2)
n1 = np.array(ex1)
n2 = np.array(ex2)

(t1*t2).numpy(), t01*t02, n1*n2

(array([[ 2.,  4.,  6.],
        [ 8., 10., 12.],
        [14., 16., 18.]], dtype=float32),
 tensor([[ 2.,  4.,  6.],
         [ 8., 10., 12.],
         [14., 16., 18.]]),
 array([[ 2.,  4.,  6.],
        [ 8., 10., 12.],
        [14., 16., 18.]]))

In [18]:
ex1 = [[1.,2.,3.],
      [4.,5.,6.],
      [7.,8.,9.]]
ex2 = [1.,2.,3.]
t1 = Tensor(ex1)
t2 = Tensor(ex2)
t01 = torch.Tensor(ex1)
t02 = torch.Tensor(ex2)
n1 = np.array(ex1)
n2 = np.array(ex2)

(t1*t2).numpy(), t01*t02, n1*n2

(array([[ 1.,  4.,  9.],
        [ 4., 10., 18.],
        [ 7., 16., 27.]], dtype=float32),
 tensor([[ 1.,  4.,  9.],
         [ 4., 10., 18.],
         [ 7., 16., 27.]]),
 array([[ 1.,  4.,  9.],
        [ 4., 10., 18.],
        [ 7., 16., 27.]]))

In [19]:
ex1 = [[2.,2.,3.],
      [4.,5.,6.],
      [7.,8.,9.]]
ex2 = [[2],
      [2],
        [3]]
t1 = Tensor(ex1)
t2 = Tensor(ex2)
t01 = torch.Tensor(ex1)
t02 = torch.Tensor(ex2)
n1 = np.array(ex1)
n2 = np.array(ex2)

(t1*t2).numpy(), t01*t02, n1*n2

(array([[ 4.,  4.,  6.],
        [ 8., 10., 12.],
        [21., 24., 27.]], dtype=float32),
 tensor([[ 4.,  4.,  6.],
         [ 8., 10., 12.],
         [21., 24., 27.]]),
 array([[ 4.,  4.,  6.],
        [ 8., 10., 12.],
        [21., 24., 27.]]))

In [20]:
ex1 = [[2.,2.,3.],
      [4.,5.,6.],
      [7.,8.,9.]]
ex2 = [2,1,1]
t1 = Tensor(ex1)
t2 = Tensor(ex2)
t01 = torch.Tensor(ex1)
t02 = torch.Tensor(ex2)
n1 = np.array(ex1)
n2 = np.array(ex2)

(t1*t2).numpy(), t01*t02, n1*n2

(array([[ 4.,  2.,  3.],
        [ 8.,  5.,  6.],
        [14.,  8.,  9.]], dtype=float32),
 tensor([[ 4.,  2.,  3.],
         [ 8.,  5.,  6.],
         [14.,  8.,  9.]]),
 array([[ 4.,  2.,  3.],
        [ 8.,  5.,  6.],
        [14.,  8.,  9.]]))

In [21]:
torch.arange(4).view(-1,2)

tensor([[0, 1],
        [2, 3]])

In [22]:
Tensor.arange(4).view(-1,2).numpy()

array([[0, 1],
       [2, 3]], dtype=int32)

In [23]:
tx = torch.arange(3).view(1, 3)
ty = torch.arange(2).view(2, 1)
tx,ty

(tensor([[0, 1, 2]]),
 tensor([[0],
         [1]]))

In [24]:
ta, tb = torch.broadcast_tensors(tx, ty) 
ta, tb

(tensor([[0, 1, 2],
         [0, 1, 2]]),
 tensor([[0, 0, 0],
         [1, 1, 1]]))

In [25]:
Tensor([False]).numpy()

array([False])

# Kernel fusion: How dot product operations work?

In [26]:
%set_env DEBUG=5
%set_env NOOPT=1

env: DEBUG=5
env: NOOPT=1


In [27]:
import tinygrad
from tinygrad import Tensor

In [46]:
a = Tensor([1,2])
b = Tensor([3,4])
at = torch.Tensor(a.numpy())
bt = torch.Tensor(b.numpy())
res = a.dot(b).numpy()
print(res)

11


In [32]:
x = Tensor([[1, 2], [3, 4]])
print(Tensor.rearrange(x, "batch channel -> (batch channel)").numpy())

AttributeError: type object 'Tensor' has no attribute 'rearrange'

In [37]:
a.dtype.scalar(), a.dtype.vec(3), a.dtype

(dtypes.int, dtypes._int3, dtypes.int)

In [43]:
from tinygrad import dtypes
dtypes.int == a.dtype.scalar()

True

In [50]:
a = Tensor([2,4])
a.dtype.count, b.dtype.count

(1, 1)

In [52]:
a.dtype.fmt

'i'